# IST 652 Final Project - A Pantheon Exploration

# Introduction

Research questions:

* Which historic characters are the most popular?
* When did they live and where are they from?
* What factors could ahve generated their popularity?
* Are there any observable trends in the categorical data provided?
* What associations exist between the variables in the data?
* What clusters and groupings are in the data? How do groups compared in popularity?

# Analysis

Key analysis methods used in report:
* Data Cleaning
* Sorting and subsets of the data.
* Line and bar plots.
* Multiple regression

## About the Data

## Data Cleaning

## Exploration

## Modeling

# Results

# Conclusion